## Purpose: Create new MIRI imager reference files for CDP7beta3 ##

First import the things that we need from the pipeline code

In [1]:
import os as os
import numpy as np
import pdb as pdb
from astropy.modeling import models
from asdf import AsdfFile
from jwst import datamodels
from jwst.assign_wcs import miri

Import the MIRI coordinates code from https://github.com/STScI-MIRI/coordinates and ensure that it is on the PYTHONPATH.  Also ensure that the output data directory is set:

setenv MIRICOORD_DATA_DIR /YourLocalPathToData/ (this is where io will happen)

In [2]:
data_dir=os.path.expandvars('$MIRICOORD_DATA_DIR')

Import the standalone MIRI imager tools that use the FITS distortion files

In [3]:
import miricoord.imager.mirim_tools as mt

Import the MIRI imager pipeline tools that can work with the CRDS distortion files

In [4]:
import miricoord.imager.mirim_pipetools as mpt

Import the code that actually creates the CRDS distortion files

In [5]:
import miricoord.imager.makecrds.makecrds_ima_cdp7beta3 as cdpcreate

Point to the IDT-delivered CDP file that we're using

In [6]:
cdpfile=mt.get_fitsreffile()

Specify our output filenames for the two asdf files

In [7]:
distfile=os.path.join(data_dir,"temp/jwst_miri_imager_distortion_cdp7b3.asdf")

In [8]:
offfile=os.path.join(data_dir,"temp/jwst_miri_filteroffset_cdp7b3.asdf")

Call the code to actually make the reference files

In [9]:
cdpcreate.make_references(cdpfile,{'DISTORTION': distfile, 'FILTEROFFSET':offfile})

Now point to these reference files and load the transform into the function 'xform'

In [10]:
refs = {"distortion": distfile, "filteroffset": offfile}

In [11]:
xform=mpt.xytov2v3model('F770W',refs=refs)

The pipeline uses 0-indexed detector pixels; as of CDP7b the MIRI ASDF object assumes this form of input and converts as necessary to the reference frames needed by various IDT transforms.  The reference point of the imager is 692.5, 511.5 in this 0-indexed convention; test that it gives the correct v2,v3 (-453.363, -374.069)

In [12]:
x,y=692.5,511.5

In [13]:
v2,v3=xform(x,y)

In [14]:
print(v2,v3)

-453.3633633490669 -374.0686305853809


And the inverse transform

In [15]:
xform.inverse(-453.3633633490669, -374.0686305853809)

(692.496064329321, 511.4999501761038)

This location (-453.363, -374.0686) is the agreed-upon reference point. 
Now run a full test of the transform against a table of known values using a dedicated testing script.  If no output, it worked.

In [16]:
cdpcreate.test_transform(refs=refs)

As of Apr 12 2017 we can't test the bounding box implementation directly because it is build into the pipeline, not the model class.  Therefore we must test this by actually running the pipeline on MIRI Imager test and reading the transform in from there.  This is done with the mirim_imagertest.ipynb notebook